In [1]:
%load_ext autoreload
%autoreload 2

# Set up Visualizer

Right now the visualizer needs to be launched independently of the Python code. The visualizer is a web app that runs in your browser, and is built with vite, a modern JavaScript build tool. The python code will communicate with the visualizer through a websocket connection.

To launch the visualizer, you will need to have node.js installed. 

With node.js installed, you can install the dependencies for the visualizer by running the following command in the terminal:

```bash
npm install
```

Then you can launch the (development version) visualizer with the following command:

```bash
npm run dev
```

A browser window should open automatically and you should see the visualizer.

TIP: If you are using Google Chrome, you can open the developer tools (F12) and go to the "Console" tab to see debug output from the visualizer.

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

from threepy.server import WebsocketServer
from threepy.geometry import *
from threepy.materials import *
from threepy.objects import *
from threepy.visualizer import Visualizer
import numpy as np
import time

In [ ]:
vis = Visualizer()

INFO:websockets.server:server listening on [::]:8001
INFO:websockets.server:server listening on 0.0.0.0:8001


INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of connections
INFO:root:Received message: {"type":"init","client":"visualizer"}


In [4]:
vis.server.is_running

True

# Adding the satellite

In [127]:
EARTH_RADIUS = 6371
altitude = 400
satellite = GLTF(name="satellite", path='albedo-sat-simple.glb')
await vis.add_object(satellite)

In [128]:
# Add the satellite to the scene
await vis.add_child('scene', 'satellite')

In [129]:
# Set the satellite position
pos_ECI = np.array([EARTH_RADIUS + altitude, 0, 0.0])
await vis.set_props('satellite', {'position': list(pos_ECI), 'scale': [1e-1 for i in range(3)]})

In [130]:
# await vis.set_props('camera', {'near': 1e-3})

In [6]:
# Set the sun direction
await vis.set_props('directionalLight', {'position': [0, 10000, 0]})

In [136]:
# Set the camera position
await vis.camera_controls(position=list(pos_ECI + np.array([0.5, 0, 0])), target=list(pos_ECI))

# Animations

In [144]:
from threepy.animations import *
# pos_kf = KeyframeTrack("satellite.position[y]", [0, 1, 2], [0.0, 10.0, 0.0], Interpolation.InterpolateSmooth)
t = np.linspace(0, 2 * np.pi, 31)
pos = np.zeros((len(t), 3))
pos[:, 0] = np.cos(t) * 1.0
pos[:, 1] = np.sin(t) * 500.0
pos = pos + pos_ECI
pos_kf = KeyframeTrack("satellite.position", list(t / (2 * np.pi) * 3), list(pos.flatten()), Interpolation.InterpolateSmooth, value_type='vector')
camera_pos_kf = KeyframeTrack("camera.position", list(t / (2 * np.pi) * 3), list((pos + np.array([0.5, 0, 0])).flatten()), Interpolation.InterpolateSmooth, value_type='vector')
clip = AnimationClip("satellite_clip", -1, [pos_kf, camera_pos_kf])
await vis.add_animation(clip)

In [145]:
await vis.load_animation(clip.name)

# Debugging

In [93]:
await vis.add_child('satellite', 'satellite_camera')

In [94]:
# Move the box in a circle
import numpy as np
import time
t = np.linspace(0, 2*np.pi, 301)
for i in range(len(t)):
    await vis.set_props("box", {"position": [np.cos(t[i]), np.sin(t[i]), 1]})
    time.sleep(0.002)

In [7]:
# Add material
mat = MeshLambertMaterial(color=0x0000ff, name="mat_blue")
await vis.add_material(mat)

In [8]:
# Add a sphere geometry
sphere = SphereGeometry(radius=0.5, name="sphere")
await vis.add_geometry(sphere)

In [9]:
# Add mesh
blue_sphere = SimpleMeshRef(geometry_name=sphere.name, material_name=mat.name, name="blue_sphere")
await vis.add_object(blue_sphere)

In [36]:
# Add mesh to scene and move
await vis.add_child("scene", "blue_sphere")
await vis.set_props("blue_sphere", {"position": [0, 0, 1]})

In [52]:
# Add cone with new geometry and material
green_cone = SimpleMesh(geometry=ConeGeometry(), material=MeshLambertMaterial(color=0x00ff00), name="green_cone")
await vis.add_object(green_cone, parent_name="scene")

In [70]:
import numpy as np
t = np.linspace(0, 2*np.pi, 31)
pos = np.zeros((len(t), 3))
pos[:, 0] = np.cos(t)
pos[:, 1] = np.sin(t)
pos[:, 2] = 1.0 + t / (2*np.pi) * 4

In [73]:
from threepy.animations import *
cone_kf = KeyframeTrack("green_cone.position[y]", [0, 1], [0.0, 10.0], Interpolation.InterpolateSmooth)
box_kf = KeyframeTrack("box.position", list(t / (2 * np.pi) * 3), list(pos.flatten()), Interpolation.InterpolateSmooth, value_type='vector')
clip = AnimationClip("animation_cone", -1, [cone_kf, box_kf])
await vis.add_animation(clip)

In [74]:
await vis.load_animation(clip.name)

In [89]:
satellite = GLTF(name="satellite", path='albedo-sat-simple.glb')
await vis.add_object(satellite)

In [90]:
await vis.add_child("scene", "satellite")

In [87]:
await vis.set_props("satellite", {"position": [50 * 1.2, 0, 0]})

In [97]:
await vis.camera_controls("controls", target=[0, 0, 0], position=[200, 0, 0])

In [8]:
vis.server.stop()